In [1]:
import hls4ml
from tensorflow.keras.models import load_model
from qkeras.utils import _add_supported_quantized_objects
import tensorflow as tf 
from util import create_config

root_dir  = f'models/deeppicar-stats/models/1101-000_64x64x1_0.1'
model_name = f'2-1101-000-0.1-100-30-50-70.h5'
model_name = f'{root_dir}/{model_name}'

co = {}
_add_supported_quantized_objects(co)
model = load_model(model_name, custom_objects=co)

2025-05-04 16:55:53.609588: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-04 16:55:53.610895: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-04 16:55:53.632547: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-04 16:55:53.633012: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-04 16:55:53.952314: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

## Print Model Summary

In [2]:
model.summary()

Model: "sequential_35"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 q_activation_139 (QActivati  (None, 64, 64, 1)        0         
 on)                                                             
                                                                 
 conv1 (QConv2D)             (None, 30, 30, 2)         52        
                                                                 
 q_activation_140 (QActivati  (None, 30, 30, 2)        0         
 on)                                                             
                                                                 
 q_conv2d_140 (QConv2D)      (None, 13, 13, 4)         204       
                                                                 
 q_activation_141 (QActivati  (None, 13, 13, 4)        0         
 on)                                                             
                                                     

In [3]:
### Create HLS Model
directory = '../best/best_model_prj'
config, hls_model = create_config(model, output_dir=directory)
print(config)

Interpreting Sequential
Topology:
Layer name: input_1, layer type: InputLayer, input shapes: [[None, 64, 64, 1]], output shape: [None, 64, 64, 1]
Layer name: q_activation_139, layer type: Activation, input shapes: [[None, 64, 64, 1]], output shape: [None, 64, 64, 1]
Layer name: conv1, layer type: QConv2D, input shapes: [[None, 64, 64, 1]], output shape: [None, 30, 30, 2]
Layer name: q_activation_140, layer type: Activation, input shapes: [[None, 30, 30, 2]], output shape: [None, 30, 30, 2]
Layer name: q_conv2d_140, layer type: QConv2D, input shapes: [[None, 30, 30, 2]], output shape: [None, 13, 13, 4]
Layer name: q_activation_141, layer type: Activation, input shapes: [[None, 13, 13, 4]], output shape: [None, 13, 13, 4]
Layer name: q_conv2d_141, layer type: QConv2D, input shapes: [[None, 13, 13, 4]], output shape: [None, 5, 5, 5]
Layer name: q_activation_142, layer type: Activation, input shapes: [[None, 5, 5, 5]], output shape: [None, 5, 5, 5]
Layer name: q_conv2d_143, layer type: QCo

In [4]:
hls_model.compile()

Writing HLS project
Done


In [5]:
### Check to see if it has the similar error
from get_dataset import get_dataset
from util import calculate_accuracy
import numpy as np

imgs_train, imgs_test, vals_train, vals_test = get_dataset('./deeppicar')

preds = hls_model.predict(imgs_test.astype(np.float32))
vals_test = np.array(vals_test)

# mse
print(f'HLS MSE: {np.mean(((preds.reshape(-1) - vals_test.reshape(-1)) ** 2))}')
print(f'Accuracy: {calculate_accuracy(preds, vals_test)}')

preds = model.predict(imgs_test.astype(np.float32))
vals_test = np.array(vals_test)

# mse
print(f'QKeras MSE: {np.mean(((preds.reshape(-1) - vals_test.reshape(-1)) ** 2))}')
print(f'Accuracy: {calculate_accuracy(preds, vals_test)}')

HLS MSE: 0.03684065863490105
Accuracy: 0.8472
79/79 [==============================] - 0s 1ms/step
QKeras MSE: 0.03689606487751007
Accuracy: 0.8472


## Build Accelerator

In [6]:
hls_model.build(csim=False)


****** Vitis HLS - High-Level Synthesis from C, C++ and OpenCL v2024.2 (64-bit)
  **** SW Build 5238294 on Nov  8 2024
  **** IP Build 5239520 on Sun Nov 10 16:12:51 MST 2024
  **** SharedData Build 5239561 on Fri Nov 08 14:39:27 MST 2024
  **** Start of session at: Sun May  4 16:56:15 2025
    ** Copyright 1986-2022 Xilinx, Inc. All Rights Reserved.
    ** Copyright 2022-2024 Advanced Micro Devices, Inc. All Rights Reserved.

source /opt/Xilinx/Vitis/2024.2/scripts/vitis_hls/hls.tcl -notrace
INFO: [HLS 200-10] For user 'petertso' on host 'pop-os' (Linux_x86_64 version 6.9.3-76060903-generic) on Sun May 04 16:56:16 CDT 2025
INFO: [HLS 200-10] On os Pop!_OS 22.04 LTS
INFO: [HLS 200-10] In directory '/home/petertso/Documents/753-FPGA/best/best_model_prj'
Sourcing Tcl script 'build_prj.tcl'
INFO: [HLS 200-1510] Running: open_project myproject_prj 
INFO: [HLS 200-10] Opening project '/home/petertso/Documents/753-FPGA/best/best_model_prj/myproject_prj'.
INFO: [HLS 200-1510] Running: set_to

{'CSynthesisReport': {'TargetClockPeriod': '30.00',
  'EstimatedClockPeriod': '20.557',
  'BestLatency': '225244',
  'WorstLatency': '225290',
  'IntervalMin': '16386',
  'IntervalMax': '225282',
  'BRAM_18K': '32',
  'DSP': '17',
  'FF': '10368',
  'LUT': '15817',
  'URAM': '0',
  'AvailableBRAM_18K': '100',
  'AvailableDSP': '90',
  'AvailableFF': '41600',
  'AvailableLUT': '20800',
  'AvailableURAM': '0'}}

## Read Vitis Report

In [27]:
hls4ml.report.read_vivado_report('tmp/tmp_prj')

Found 1 solution(s) in tmp/tmp_prj/myproject_prj.
Reports for solution "solution1":

C simulation report not found.
SYNTHESIS REPORT:
== Vitis HLS Report for 'myproject'
* Date:           Sun May  4 16:34:26 2025

* Version:        2024.2 (Build 5238294 on Nov  8 2024)
* Project:        myproject_prj
* Solution:       solution1 (Vivado IP Flow Target)
* Product family: artix7
* Target device:  xc7a35t-cpg236-1


== Performance Estimates
+ Timing: 
    * Summary: 
    +--------+----------+-----------+------------+
    |  Clock |  Target  | Estimated | Uncertainty|
    +--------+----------+-----------+------------+
    |ap_clk  |  30.00 ns|  20.557 ns|     8.10 ns|
    +--------+----------+-----------+------------+

+ Latency: 
    * Summary: 
    +---------+---------+----------+----------+-------+--------+----------+
    |  Latency (cycles) |  Latency (absolute) |    Interval    | Pipeline |
    |   min   |   max   |    min   |    max   |  min  |   max  |   Type   |
    +---------+-----

In [ ]:
from get_dataset import get_dataset

imgs, vals = get_dataset('./deeppicar', return_as_is=True)

# take the first image and create 
img = imgs[0]

with open('data.hex', 'w') as f:
    for d in img.flatten():
        f.write(f'{d:02X}\n')

[array([[  1,   1,   0, ...,   6,  11,  19],
       [  1,   1,   0, ...,  97,  57,  40],
       [  0,   0,   1, ..., 143, 135, 112],
       ...,
       [ 94,  94,  94, ..., 108, 109, 110],
       [ 94,  94,  94, ..., 108, 109, 110],
       [ 95,  95,  94, ..., 109, 108, 107]], dtype=uint8), array([[  0,   0,   0, ...,   6,   9,  22],
       [  0,   0,   0, ...,  97,  55,  39],
       [  1,   1,   0, ..., 146, 133, 113],
       ...,
       [ 94,  94,  94, ..., 107, 108, 109],
       [ 94,  94,  94, ..., 107, 109, 110],
       [ 94,  93,  94, ..., 108, 107, 107]], dtype=uint8), array([[  0,   0,   0, ...,   5,  10,  23],
       [  0,   0,   0, ...,  96,  56,  40],
       [  0,   0,   0, ..., 146, 133, 113],
       ...,
       [ 94,  94,  94, ..., 107, 108, 109],
       [ 94,  94,  94, ..., 108, 109, 110],
       [ 94,  93,  94, ..., 105, 106, 107]], dtype=uint8), array([[  0,   0,   0, ...,   5,  10,  23],
       [  0,   0,   0, ...,  96,  56,  40],
       [  0,   0,   0, ..., 147, 133, 